In [1]:
# !pip install transformers torch ipywidgets pillow

# I think cybershuttle probably has this already, but it doesn't hurt to have

In [2]:
# Imports
import torch
from PIL import Image
from ipywidgets import FileUpload, Button, VBox, Output, Label
from IPython.display import display
import io
import os
import time
from transformers import AutoModel, AutoFeatureExtractor, AutoImageProcessor, ResNetForImageClassification
from PIL import Image
import tracemalloc
import subprocess
import gc
import platform

2025-04-26 21:13:23.553000: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 21:13:23.666085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745716403.711374   11754 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745716403.727108   11754 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 21:13:23.846515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
model_name = "microsoft/resnet-50"

# Download model and feature extractor (if needed)
save_directory = "./my_local_model"
if not os.path.exists(save_directory):
    model = ResNetForImageClassification.from_pretrained(model_name)  # Use the correct class
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

    # Save locally
    model.save_pretrained(save_directory)
    feature_extractor.save_pretrained(save_directory)
    print("Model and feature extractor saved successfully!")
else:
    # Load from saved directory
    model = ResNetForImageClassification.from_pretrained(save_directory)
    feature_extractor = AutoFeatureExtractor.from_pretrained(save_directory)
    print("Loaded model from local directory")

# Move model to appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/home/pkadekodi/.local/lib/python3.10/site-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


Model and feature extractor saved successfully!


In [4]:
# # Load model and processor (only once for optimization purposes)
# processor = AutoImageProcessor.from_pretrained("./my_local_model")
# model = ResNetForImageClassification.from_pretrained("./my_local_model")

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)

In [5]:
def classify_image(image, feature_extractor, model):
    if not isinstance(image, Image.Image):
        image = Image.open(image)

    inputs = feature_extractor(image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_label = logits.argmax(-1).item()
    return model.config.id2label[predicted_label]

In [6]:
# UI Setup (multiple files accepted)
uploader = FileUpload(accept='image/*', multiple=True)
classify_button = Button(description="🚀 Classify Image", button_style="success")
clear_output_button = Button(description="🧹 Clear Results", button_style="warning")
status_label = Label("Upload an image and click the button!")
output = Output()

# Button logic with "Thinking..." status
def on_classify_clicked(b):
    output.clear_output()
    status_label.value = "🧠 Thinking..."

    # Loop through all images uploaded
    if uploader.value:
        with output:
            for uploaded_filename in uploader.value:
                uploaded_content = uploader.value[uploaded_filename]['content']
                image = Image.open(io.BytesIO(uploaded_content))

                prediction = classify_image(image, feature_extractor, model)

                display(image)
                print(f"🧠 Predicted class for {uploaded_filename}: {prediction}")

        status_label.value = "✅ Done!"
    else:
        with output:
            print("❗ Please upload at least one image.")
        status_label.value = "⚠️ No images uploaded."

# Clear output function
def on_clear_output_clicked(b):
    uploader._counter = 0
    output.clear_output()
    status_label.value = "Upload an image and click the button!"
    with output:
        print("Results cleared")

# Connect buttons to functions
classify_button.on_click(on_classify_clicked)
clear_output_button.on_click(on_clear_output_clicked)

# Launch app
display(VBox([status_label, uploader, classify_button, clear_output_button, output]))